In [1]:
# Import necessary libraries
import pandas as pd
import logging

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Datastore

ModuleNotFoundError: No module named 'azureml'

In [6]:
df_nyc = pd.read_csv("all_features.csv") # Change the file path to the dataset if not
df_nyc.head(5)

,Unnamed: 0,date,count,Year,Month,Week,Day,Day of Week,Day of Year,traffic_volume,...,day_26,day_27,day_28,day_29,day_30,day_31,holiday,PRCP_log,SNOW_log,SNWD_log
0,0,2014-01-01,6059.0,2014,1,1,1,2,1,497752.0,...,0,0,0,0,0,0,1,0.000000,0.000000,0.000000
1,31,2014-01-02,8600.0,2014,1,1,2,3,2,526682.0,...,0,0,0,0,0,0,0,0.285179,1.410987,0.000000
2,59,2014-01-03,1144.0,2014,1,1,3,4,3,258287.0,...,0,0,0,0,0,0,0,0.254642,1.458615,1.931521
3,90,2014-01-04,2292.0,2014,1,1,4,5,4,474493.0,...,0,0,0,0,0,0,0,0.000000,0.000000,1.931521
4,120,2014-01-05,2678.0,2014,1,1,5,6,5,454609.0,...,0,0,0,0,0,0,0,0.131028,0.000000,1.589235


In [7]:
df_nyc.drop(columns = {"Unnamed: 0"}, inplace = True)

In [8]:
df_nyc.drop(columns = {"date"}, inplace = True)

In [9]:
train_data = df_nyc[df_nyc["Year"] != 2021]
test_data =  df_nyc[df_nyc["Year"] == 2021]

In [10]:
# Logging into azure
# Configure workspace and experiment
ws = Workspace.from_config() #This will automatically look for the config.json that has the keys for sign in Azure. 
exp = Experiment(workspace=ws, name="bike_sharing_6_12") #Name the project 

NameError: name 'Workspace' is not defined

In [12]:
experiment_name = "bike_sharing_6_12"

In [13]:
output = {}
output["Subscription ID"] = ws.subscription_id
output["Workspace"] = ws.name
output["Resource Group"] = ws.resource_group
output["Location"] = ws.location
output["Run History Name"] = experiment_name
output["SDK Version"] = azureml.core.VERSION
pd.set_option("display.max_colwidth", None)
outputDf = pd.DataFrame(data=output, index=[""])
outputDf.T


NameError: name 'ws' is not defined

In [2]:
# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster exist already. If not, create computer clusters
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_D2_V2", max_nodes=20
    )
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

NameError: name 'ComputeTargetException' is not defined

In [3]:
# set up datastores
dstore = ws.get_default_datastore()

NameError: name 'ws' is not defined

In [4]:
# Upload the train dataset into the cloud
datastore = ws.get_default_datastore()
dataset = Dataset.Tabular.register_pandas_dataframe(train_data, datastore, "6_12_train_transformed")

NameError: name 'ws' is not defined

In [5]:
# Upload the test dataset into the cloud
datastore = ws.get_default_datastore()
dataset_test = Dataset.Tabular.register_pandas_dataframe(test_data, datastore, "6_12_test_transformed")

NameError: name 'ws' is not defined

In [13]:
# Label in the dataset for dependent variables
label = "count"

In [14]:
# Setting the autoML
automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": "r2_score",
    "enable_early_stopping": True,
    "experiment_timeout_hours": 1,  # for real scenarios we reccommend a timeout of at least one hour
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
    "verbosity": logging.INFO,
}
automl_config = AutoMLConfig(
    task="regression",
    compute_target=compute_target,
    training_data=dataset,
    label_column_name=label,
    **automl_settings,
)

In [15]:
# Submit the job and go to Details Page for detailed results
remote_run = exp.submit(automl_config, show_output=False)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
bike_sharing_6_12,AutoML_4621d232-2939-487f-9c6e-c1f060cda9a9,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation
